In [ ]:
from matplotlib import pyplot as plt
import os
import pandas as pd
import numpy as np
import seaborn as sns
from torchvision.transforms import v2

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation, SegformerImageProcessor
from PIL import Image
import requests
import torch

feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/segformer-b0-finetuned-cityscapes-512-1024")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b0-finetuned-cityscapes-512-1024")
processor = SegformerImageProcessor.from_pretrained("nvidia/segformer-b0-finetuned-cityscapes-512-1024")

Description
---

- Use the pretrained NVIDIA Segformer model to classify pixels
- Select only class 2
- Save image, masked pixels filled with 0 --> street_segm0.jpg
- Create bounding box containing the class 2 pixels, save image --> street_segmbbox.jpg

In [ ]:
data_root = '/work/ka1176/caroline/gitlab/AI4EO-MapYourCity/data/AI4EO-MapYourCity/v1/building-age-dataset'

In [ ]:
split = 'test'
data_path = os.path.join(data_root, split, 'data')

In [ ]:
df = pd.read_csv(os.path.join(data_root, split, f'{split}-set.csv'))
df.head()

In [ ]:
source_filename = 'street.jpg'
target_filename = 'street_segm0.jpg'
target_bb_filename = 'street_segmbox.jpg'
target_patch_filename = 'street_patch_tiny.jpg'
mask_filename = 'is_house.npy'

In [ ]:
overwrite = False

In [ ]:
problem_pids = ['bdpfdxr3yo', 'g3gzdwy6t8']

In [ ]:
min_frac = 0.85
scale = (0.08, 0.25)

In [ ]:
for pid in df['pid'].values[::-1]:
    #if pid in problem_pids:
    #    continue
    
    if not os.path.exists(os.path.join(data_path, pid, source_filename)):
        print('missing source', pid)
        continue
    
    if os.path.exists(os.path.join(data_path, pid, target_filename)) and \
       os.path.exists(os.path.join(data_path, pid, target_patch_filename)):
        if not overwrite: continue    
    
    with open(os.path.join(data_path, pid, source_filename), 'rb') as ff:
        image = Image.open(ff)
        image.load()
        image = image.convert("RGB")
        raw = plt.imread(os.path.join(data_path, pid, source_filename))
        
        # Segment
        inputs = feature_extractor(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model(**inputs)
            
        # Get predicted map
        predicted_segmentation_map = processor.post_process_semantic_segmentation(outputs, target_sizes=[image.size[::-1]])[0]
        predicted_segmentation_map = predicted_segmentation_map.cpu().numpy()
        
        # Mask: Keep only class 2 (which I assume is "house")
        if not 2 in np.unique(predicted_segmentation_map):
            print(f'Class not found in PID {pid}: Keep original image')
            image.save(os.path.join(data_path, pid, target_filename))
            image.save(os.path.join(data_path, pid, target_bb_filename))
            image.save(os.path.join(data_path, pid, target_patch_filename))
            #mask = np.ma.make_mask(predicted_segmentation_map==2)
            #mask[:,:] = 1
            #np.save(os.path.join(data_path, pid, mask_filename), mask)
            
            continue
        mask = np.ma.make_mask(predicted_segmentation_map==2)
        np.save(os.path.join(data_path, pid, mask_filename), mask)
        masked_raw = np.where(np.dstack([mask, mask, mask]), raw, 0)
        
        # Sample random patch
        img = plt.imread(os.path.join(data_path, pid, 'street.jpg'))
        
        keep_sampling = True
        s=0
        stacked = torch.dstack((torch.Tensor(img), torch.Tensor(mask).unsqueeze(-1)))
        stacked = stacked.transpose(0, 2).transpose(1, 2)   
        
        while keep_sampling:
            rrcrop = v2.RandomResizedCrop(384, scale=scale, interpolation=3)
            timg = rrcrop(stacked).numpy().transpose(1,2,0)   
            crmask = timg[:,:,-1]
            frac = crmask.sum()/np.prod(crmask.shape)
            s+=1
            
            if frac > min_frac or s > 25:
                keep_sampling = False
                
        timg = timg[:,:,:3]
        
        patch = Image.fromarray(timg.astype(np.uint8))
        patch.save(os.path.join(data_path, pid, target_patch_filename))
        print(f'House fraction {frac:.2f} in {s} tries')
        
        # Bounding box encompassing the house in the segmentation map
        segmentation = np.where(mask==True)
        x_min = int(np.min(segmentation[1]))
        x_max = int(np.max(segmentation[1]))
        y_min = int(np.min(segmentation[0]))
        y_max = int(np.max(segmentation[0]))
        bb = raw[y_min:y_max, x_min:x_max]
                
        # Save as image
        masked_image = Image.fromarray(masked_raw)
        masked_image.save(os.path.join(data_path, pid, target_filename))
        
        bbox_image = Image.fromarray(bb)
        bbox_image.save(os.path.join(data_path, pid, target_bb_filename))
        print(f'Targets created for PID {pid}')

In [ ]:
fig, ax = plt.subplots(2, 3, sharex=False, sharey=False, figsize=(10, 10))
ax = ax.flatten()
pid = 'mxgek8kuqt'
ax[0].imshow(plt.imread(os.path.join(data_path, pid, source_filename)))
ax[1].imshow(plt.imread(os.path.join(data_path, pid, target_filename)));
ax[2].imshow(plt.imread(os.path.join(data_path, pid, target_bb_filename)));
ax[3].imshow(plt.imread(os.path.join(data_path, pid, target_patch_filename)));
ax[4].imshow(np.load(os.path.join(data_path, pid, mask_filename)));